# Install requirements

In [23]:
!pip install -r requirements.txt

In [2]:
# Save model to OpenVINO format

from optimum.intel.openvino import OVModelForVisualCausalLM

# First time: export and save
model = OVModelForVisualCausalLM.from_pretrained(
    "llava-hf/LLaVA-NeXT-Video-7B-hf", 
    export=True,
    trust_remote_code=True
)

model.save_pretrained("./llava_openvino_model")

# Future times: load from local saved version (much faster)
model = OVModelForVisualCausalLM.from_pretrained("./llava_openvino_model")

/home/ubuntu/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 26.03it/s]
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Using a slow image processor

# Upload OV model to HF 

In [11]:
# Create model card and upload to Hugging Face Hub

from huggingface_hub import HfApi, create_repo
import os

# Replace with your desired repo name
REPO_NAME = "llava-next-video-openvino"  # Change this to your preferred name
HF_USERNAME = "ezelanza"  # Replace with your HF username

# Create model card content
model_card = """---
license: apache-2.0
base_model: llava-hf/LLaVA-NeXT-Video-7B-hf
tags:
- openvino
- llava
- multimodal
- video
- visual-question-answering
---

# LLaVA-NeXT-Video OpenVINO Model

This is an OpenVINO optimized version of the LLaVA-NeXT-Video-7B-hf model.

## Model Description
- **Base Model**: llava-hf/LLaVA-NeXT-Video-7B-hf
- **Optimization**: Converted to OpenVINO format for efficient inference
- **Size**: ~7B parameters

## Usage

```python
from optimum.intel.openvino import OVModelForVisualCausalLM

model = OVModelForVisualCausalLM.from_pretrained("YOUR_USERNAME/llava-next-video-openvino")
```

## License
This model inherits the license from the original LLaVA-NeXT model.
"""

# Save model card
with open("README.md", "w") as f:
    f.write(model_card)

print("Model card created: README.md")


Model card created: README.md


In [14]:
# Upload model to Hugging Face Hub

from huggingface_hub import HfApi
import os
# Login to Hugging Face

from huggingface_hub import login
import getpass

print("Go to: https://huggingface.co/settings/tokens")
print("Create a new token with WRITE permissions")
print()

token = getpass.getpass("Enter your HF token: ")
login(token=token)

# Configuration - UPDATE THESE VALUES
REPO_NAME = "ezelanza/llava-next-video-openvino"  # Your desired repo name
# The username will be automatically detected from your login

api = HfApi()

# Create repository
try:
    repo_url = api.create_repo(
        repo_id=REPO_NAME,
        exist_ok=True,
        repo_type="model"
    )
    print(f"Repository created/exists: {repo_url}")
except Exception as e:
    print(f"Repository creation error: {e}")

# Upload model files if they exist
if os.path.exists("./llava_openvino_model"):
    print("Uploading model files...")
    api.upload_folder(
        folder_path="./llava_openvino_model",
        repo_id=REPO_NAME,
        repo_type="model"
    )
    
    # Upload README
    if os.path.exists("README.md"):
        api.upload_file(
            path_or_fileobj="README.md",
            path_in_repo="README.md",
            repo_id=REPO_NAME,
            repo_type="model"
        )
    
    print(f"✅ Model uploaded successfully!")
    print(f"🔗 View your model at: https://huggingface.co/{api.whoami()['name']}/{REPO_NAME}")
else:
    print("❌ Model directory './llava_openvino_model' not found.")
    print("Run the first cell to save the model first.")


Go to: https://huggingface.co/settings/tokens
Create a new token with WRITE permissions

Repository created/exists: https://huggingface.co/ezelanza/llava-next-video-openvino
Uploading model files...


openvino_language_model.bin:   0%|          | 0.00/26.2G [00:00<?, ?B/s]









openvino_vision_resampler_model.bin: 100%|██████████| 100/100 [00:00<00:00, 1.85kB/s]
openvino_language_model.bin:   0%|          | 9.37M/26.2G [00:00<04:39, 93.6MB/s]








openvino_language_model.bin:   0%|          | 26.2M/26.2G [00:00<12:48, 34.0MB/s]


openvino_language_model.bin:   0%|          | 52.4M/26.2G [00:01<07:52, 55.3MB/s]


openvino_language_model.bin:   0%|          | 78.5M/26.2G [00:01<06:22, 68.3MB/s]


openvino_multi_modal_projector_model.bin: 100%|██████████| 83.9M/83.9M [00:01<00:00, 57.0MB/s]


openvino_language_model.bin:   0%|          | 105M/26.2G [00:01<05:46, 75.3MB/s] 



openvino_language_model.bin:   1%|          | 131M/26.2G [00:01<05:46, 75.0MB/s]

openvino_language_model.bin:   1%|          | 155M/26.2G [00:02<04:32, 95.6MB/s]


openvino_language_model.bin:   1%|          | 168M/26.2G [00:02<05:25, 79.9MB/s]



openvino_language_model.bin:   1%|          | 183M/26.2G [0

✅ Model uploaded successfully!
🔗 View your model at: https://huggingface.co/ezelanza/ezelanza/llava-next-video-openvino


# Load the model 

In [18]:
model_id = "ezelanza/llava-next-video-openvino"
from transformers import LlavaNextVideoProcessor

model = OVModelForVisualCausalLM.from_pretrained(model_id)
processor = LlavaNextVideoProcessor.from_pretrained("llava-hf/LLaVA-NeXT-Video-7B-hf")


# Optimize it

In [1]:
from optimum.intel import OVQuantizationConfig, OVWeightQuantizationConfig, OVPipelineQuantizationConfig

dataset, num_samples = "contextual", 50

# weight-only 8bit
woq_8bit = OVWeightQuantizationConfig(bits=8)

# weight-only 4bit
woq_4bit = OVWeightQuantizationConfig(bits=4, group_size=16)

# static quantization
static_8bit = OVQuantizationConfig(bits=8, dataset=dataset, num_samples=num_samples)

# pipeline quantization: applying different quantization on each components
ppl_q = OVPipelineQuantizationConfig(
    quantization_configs={
        "lm_model": OVQuantizationConfig(bits=8),
        "multimodal_model": OVWeightQuantizationConfig(bits=8),
        "text_embeddings_model": OVWeightQuantizationConfig(bits=8),
        "vision_embeddings_model": OVWeightQuantizationConfig(bits=8),
        "vision_model": OVWeightQuantizationConfig(bits=8) 
    },
    dataset=dataset,
    num_samples=num_samples,
)

/home/ubuntu/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from optimum.intel import OVModelForVisualCausalLM, OVWeightQuantizationConfig

model_id = "ezelanza/llava-next-video-openvino"

q_model = OVModelForVisualCausalLM.from_pretrained(model_id, quantization_config=woq_8bit)
int8_model_path = "llava_next_video_int8"
q_model.save_pretrained(int8_model_path)

INFO:nncf:Statistics of the bitwidth distribution:
┍━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┑
│ Weight compression mode   │ % all parameters (layers)   │ % ratio-defining parameters (layers)   │
┝━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┥
│ int8_asym                 │ 100% (225 / 225)            │ 100% (225 / 225)                       │
┕━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┙


/home/ubuntu/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

INFO:nncf:Statistics of the bitwidth distribution:
┍━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┑
│ Weight compression mode   │ % all parameters (layers)   │ % ratio-defining parameters (layers)   │
┝━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┥
│ int8_sym                  │ 100% (1 / 1)                │ 100% (1 / 1)                           │
┕━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┙


INFO:nncf:Statistics of the bitwidth distribution:
┍━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┑
│ Weight compression mode   │ % all parameters (layers)   │ % ratio-defining parameters (layers)   │
┝━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┥
│ int8_sym                  │ 100% (139 / 139)            │ 100% (139 / 139)                       │
┕━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┙


INFO:nncf:Statistics of the bitwidth distribution:
┍━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┑
│ Weight compression mode   │ % all parameters (layers)   │ % ratio-defining parameters (layers)   │
┝━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┥
┕━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┙
INFO:nncf:Statistics of the bitwidth distribution:
┍━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┑
│ Weight compression mode   │ % all parameters (layers)   │ % ratio-defining parameters (layers)   │
┝━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┥
│ int8_sym                  │ 100% (2 / 2)                │ 100% (2 / 2)                           │
┕━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Run inference

In [2]:
from huggingface_hub import hf_hub_download 
from transformers import LlavaNextVideoProcessor
from optimum.intel.openvino import OVModelForVisualCausalLM

video_path = hf_hub_download(repo_id="raushan-testing-hf/videos-test", filename="sample_demo_1.mp4", repo_type="dataset")

conversation = [
    {

        "role": "user",
        "content": [
            {"type": "text", "text": "What is happening in the video?"},
            {"type": "video", "path": video_path},
            ],
    },
]
processor = LlavaNextVideoProcessor.from_pretrained("llava-hf/LLaVA-NeXT-Video-7B-hf")
model = OVModelForVisualCausalLM.from_pretrained("./llava_next_video_int8")


inputs = processor.apply_chat_template(
    conversation,
    num_frames=8,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
)

# 🚀 Step 5: Run inference
out = model.generate(**inputs, max_new_tokens=60)

# 🧾 Step 6: Decode output
response = processor.batch_decode(
    out,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True
)[0]

print(response)

Unused or unrecognized kwargs: return_tensors.


USER: 
What is happening in the video? ASSISTANT: In the video, we see a young child sitting on a bed, wearing glasses and holding a book. The child appears to be engaged in reading or looking at the book, possibly turning the pages. The room has a cozy and lived-in feel, with various items like a bed
